## Installing libs

In [80]:
! pip install transformers
! pip install huggingface

!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

!apt install git-lfs

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2537  100  2537    0     0   130k      0 --:--:-- --:--:-- --:--:--  130k
OK
62 packages can be upgraded. Run 'apt list --upgradable' to see them.
gcsfuse is already the newest version (0.39.2).
The following packages were automatically installed and are no longer required:
  cuda-command-line-tools-10-0 cuda-command-line-tools-10-1
  cuda-command-line-tools-11-0 cuda-compiler-10-0 cuda-compiler-10-1
  cuda-compiler-11-0 cuda-cuobjdump-10-0 cuda-cuobjdump-10-1
  cuda-cuobjdump-11-0 cuda-cupti-10-0 cuda-cupti-10-1 cuda-cupti-11-0
  cuda-cupti-dev-11-0 cuda-documentation-10-0 cuda-documentation-10-1
  cuda-documentation-11-0 cuda-documentation-11-1 cuda-gdb-10-0 cuda-gdb-10-1
  cuda-gdb-11-0 cuda-gpu-library-advisor-10-0 cuda-gpu-library-advisor-10-1
  cuda-libraries-10-0 cuda-libraries-10-1 cuda-libraries-11-0
  cuda-memcheck-

## Vars

In [45]:
project_id = 'northern-net-316320'
bucket_name = 'generic-projects'


## huggingface login

In [ ]:
!huggingface-cli login


## training

In [93]:
from transformers import TextDataset,DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments,AutoModelWithLMHead
from transformers import AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelWithLMHead.from_pretrained("gpt2")


def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)
     
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

from transformers import Trainer, TrainingArguments,AutoModelWithLMHead



training_args = TrainingArguments(
    output_dir="/content/"+bucket_name+"/TextCortex/models/gpt2-mod", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=2, # number of training epochs
    prediction_loss_only=True,
    )

train_path="/content/"+bucket_name+"/TextCortex/data/train.txt"
test_path="/content/"+bucket_name+"/TextCortex/data/test.txt"
train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

trainer.train()


Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/gpt2/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/fc674cd6907b4c9e933cb42d67662436b89fa9540a1f40d7c919d0109289ad01.7d2e0efa5ca20cef4fb199382111e9d3ad96fd77b849e1d4bed13a66e1336f51
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": 

Step,Training Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=22, training_loss=3.75686298717152, metrics={'train_runtime': 7.5077, 'train_samples_per_second': 21.844, 'train_steps_per_second': 2.93, 'total_flos': 10712973312000.0, 'train_loss': 3.75686298717152, 'epoch': 2.0})

## pushing the model

In [97]:
model.push_to_hub("vwoloszyn/gtp2-email")
trainer.save_model()

/usr/local/lib/python3.7/dist-packages/huggingface_hub/hf_api.py:1004: FutureWarning: `create_repo` now takes `token` as an optional positional argument. Be sure to adapt your code!
  FutureWarning,
Cloning https://huggingface.co/vwoloszyn/gtp2-email into local empty directory.
Configuration saved in vwoloszyn/gtp2-email/config.json
Model weights saved in vwoloszyn/gtp2-email/pytorch_model.bin


Upload file pytorch_model.bin:   0%|          | 3.39k/487M [00:00<?, ?B/s]

To https://huggingface.co/vwoloszyn/gtp2-email
   b2a70c7..81f981f  main -> main



'https://huggingface.co/vwoloszyn/gtp2-email/commit/81f981f0e0de4c4df226bdc8f703f58a0f40d53c'

## Testing

In [70]:
from transformers import GPT2Tokenizer, GPT2Model
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer

#tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelWithLMHead.from_pretrained("/content/generic-projects/TextCortex/models/gpt2-mod/", pad_token_id=tokenizer.eos_token_id)


/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:807: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
loading configuration file /content/generic-projects/TextCortex/models/gpt2-mod/config.json
Model config GPT2Config {
  "_name_or_path": "/content/generic-projects/TextCortex/models/gpt2-mod/",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "pad_token_id": 50256,
  "reorder_and_upcast_attn": false,
  

In [24]:

prompt = "From: Me \n To: Vinicius \n Subject: I want to have dinner with you\n\nDear,\n\n"

input_ids = tokenizer(prompt, return_tensors="pt").input_ids

gen_tokens = model.generate(
   input_ids, 
    max_length=128, 
    no_repeat_ngram_size=2, 
    num_return_sequences=1, 
    early_stopping=True
)
gen_text = tokenizer.batch_decode(gen_tokens)
gen_text

['From: Me \n To: Vinicius \n Subject: I want to have dinner with you\n\nDear,\n\n\nI am writing to you about the recent incident in which a young man was attacked by a group of young men in a pub in the city centre.\n\n\n\n I am concerned that this is a serious incident and that the young people who attacked the man are not responsible for the attack. I would like to know if you would be willing to speak to me about this matter. Please let me know what you think.\n\n\n \n\n Sincerely,\n   Vin\nSent from my iPhone\n<|endoftext|>']